In [1]:
pip install scispacy
pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 kB 23.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 4.6 MB/s eta 0:00:00
  Using cached pybind11-2.6.1-py2.py3-none-any.whl (188 kB)
  Created wheel for nmslib: filename=nmslib-2.1.1-cp310-cp310-linux_x86_64.whl size=13572131 sha256=96e7a644c763c888c038d21091a1de1b3ed8eeb42ae66d610bd8abbb9a927ea1
  Stored in directory: /root/.cache/pip/wheels/21/1a/5d/4cc754a5b1a88405cad184b76f823897a63a8d19afcd4b9314
Successfully built nmslib
  Attempting uninstall: wasabi
    Found existing installation: wasabi 1.1.1
    Uninstalling wasabi-1.1.1:
      Successfully uninstalled wasabi-1.1.1
  Attempting uninstall: spacy


In [129]:
#connect to the ggdrive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import spacy
import scispacy
import tensorflow as tf
import itertools
import random
import numpy as np 
from scispacy.linking import EntityLinker



/Users/matthieudecotignie/opt/miniconda3/envs/RoboDoc/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-07-11 18:42:22.960665: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
Your CPU supports instructions that this binary was not compiled to use: AVX2
For maximum performance, you can install NMSLIB from sources 
pip install --no-binary :all: nmslib


In [2]:
nlp = spacy.load("en_core_sci_sm")
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})
linker = nlp.get_pipe("scispacy_linker")


/Users/matthieudecotignie/opt/miniconda3/envs/RoboDoc/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfTransformer from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/matthieudecotignie/opt/miniconda3/envs/RoboDoc/lib/python3.9/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 1.1.2 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [3]:

# Return a list of triplet tuples (UMLS concept ID, UMLS Canonical Name, Match score) for each entity that is in the given list of UMLS types
def filter_entities_by_types_group(umls_entities_list: list, type_group: list) -> list:
    return [(umls_entities_list[0][0].concept_id, umls_entities_list[0][0].canonical_name, umls_entities_list[0][1]) ]

In [4]:
def extract_umls_entities(kb_ents: list) -> list:
    # 1. T184 Sign or Symptom - return ALL and not only the first
    type_group_1 = ['T184']

    # 2. T037 Injury or Poisoning
    type_group_2 = ['T037']

    # 3. T004 Fungus | T005 Virus | T007 Bacterium | T033 Finding | T034 Laboratory or Test Result | T048 Mental or Behavioral Dysfunction
    type_group_3 = ['T004', 'T005', 'T007', 'T033', 'T034', 'T048']

    # 4. T047 Disease or Syndrome | T121 Pharmacologic Substance | T131 Hazardous or Poisonous Substance
    type_group_4 = ['T047', 'T121', 'T131']

    # Extract the UMLS Entities from each CUI value
    umls_entities_list = list(map(lambda ent: (linker.kb.cui_to_entity[ent[0]], ent[1]), kb_ents))

    type_1_matches_list = filter_entities_by_types_group(umls_entities_list, type_group_1)
    type_2_matches_list = []
    type_3_matches_list = []
    type_4_matches_list = []

    if not type_1_matches_list:
        type_2_matches_list = filter_entities_by_types_group(umls_entities_list, type_group_2)

        if not type_2_matches_list:
            type_3_matches_list = filter_entities_by_types_group(umls_entities_list, type_group_3)

        if not type_3_matches_list:
            type_4_matches_list = filter_entities_by_types_group(umls_entities_list, type_group_4)

    secondary_matches_list = list(itertools.chain(type_2_matches_list, type_3_matches_list, type_4_matches_list))

    best_fit_secondary_value = 0
    best_fit_secondary_tuple = False

    # Keep only one secondary match at the most (the one with the best match value)
    for match in secondary_matches_list:
        if match[2] > best_fit_secondary_value:
            best_fit_secondary_tuple = match
            best_fit_secondary_value = match[2]

    if best_fit_secondary_tuple:
        type_1_matches_list.append(best_fit_secondary_tuple)

    return type_1_matches_list




In [5]:

def find_similarity(complaint_str: str)->list:
    if not isinstance(complaint_str, str):
        return [], []

    umls_entity_list = []
    complaint_str_parenthesis = '"{}"'.format(complaint_str)
    doc = nlp(complaint_str_parenthesis)
    for ent in doc.ents:
        if len(ent._.kb_ents) > 0 and len(ent._.kb_ents[0]) > 0:
            most_similar_list = extract_umls_entities(ent._.kb_ents)
            umls_entity_list+=most_similar_list

    return umls_entity_list

In [128]:
def symp_to_umls(symp_arr: list)-> list:
  umls=[]
  tuples =find_similarity('/ ' +" / ".join(symp_arr)+' /' )
  for tuple in tuples:
    umls.append(tuple[0])
  return umls

In [7]:
def symp_to_umlsV2(symp_arr: list)-> list:
  cleanSymp = [sub.strip().replace('_', ' ') for sub in symp_arr]

  umls=[]
  for s in cleanSymp:
    subumls=[]
    tuples=find_similarity('/ '+s+' /')
    for tuple in tuples:
     subumls.append(tuple[0])
    umls.append(subumls)
  return umls

In [9]:
possible_symp = [' abdominal pain', ' abnormal menstruation', ' acidity', ' acute_liver_failure', ' altered_sensorium', ' anxiety', ' back_pain', ' belly_pain', ' blackheads', ' bladder_discomfort', ' blister', ' blood_in_sputum', ' bloody_stool', ' blurred_and_distorted_vision', ' breathlessness', ' brittle_nails', ' bruising', ' burning_micturition', ' chest_pain', ' chills', ' cold_hands_and_feets', ' coma', ' congestion', ' constipation', ' continuous_feel_of_urine', ' continuous_sneezing', ' cough', ' cramps', ' dark_urine', ' dehydration', ' depression', ' diarrhoea', ' dischromic _patches', ' distention_of_abdomen', ' dizziness', ' drying_and_tingling_lips', ' enlarged_thyroid', ' excessive_hunger', ' extra_marital_contacts', ' family_history', ' fast_heart_rate', ' fatigue', ' fluid_overload', ' foul_smell_of urine', ' headache', ' high_fever', ' hip_joint_pain', ' history_of_alcohol_consumption', ' increased_appetite', ' indigestion', ' inflammatory_nails', ' internal_itching', ' irregular_sugar_level', ' irritability', ' irritation_in_anus', ' joint_pain', ' knee_pain', ' lack_of_concentration', ' lethargy', ' loss_of_appetite', ' loss_of_balance', ' loss_of_smell', ' malaise', ' mild_fever', ' mood_swings', ' movement_stiffness', ' mucoid_sputum', ' muscle_pain', ' muscle_wasting', ' muscle_weakness', ' nausea', ' neck_pain', ' nodal_skin_eruptions', ' obesity', ' pain_behind_the_eyes', ' pain_during_bowel_movements', ' pain_in_anal_region', ' painful_walking', ' palpitations', ' passage_of_gases', ' patches_in_throat', ' phlegm', ' polyuria', ' prominent_veins_on_calf', ' puffy_face_and_eyes', ' pus_filled_pimples', ' receiving_blood_transfusion', ' receiving_unsterile_injections', ' red_sore_around_nose', ' red_spots_over_body', ' redness_of_eyes', ' restlessness', ' runny_nose', ' rusty_sputum', ' scurring', ' shivering', ' silver_like_dusting', ' sinus_pressure', ' skin_peeling', ' skin_rash', ' slurred_speech', ' small_dents_in_nails', ' spinning_movements', ' spotting_ urination', ' stiff_neck', ' stomach_bleeding', ' stomach_pain', ' sunken_eyes', ' sweating', ' swelled_lymph_nodes', ' swelling_joints', ' swelling_of_stomach', ' swollen_blood_vessels', ' swollen_extremeties', ' swollen_legs', ' throat_irritation', ' toxic_look_(typhos)', ' ulcers_on_tongue', ' unsteadiness', ' visual_disturbances', ' vomiting', ' watering_from_eyes', ' weakness_in_limbs', ' weakness_of_one_body_side', ' weight_gain', 'weight loss', ' yellow crust ooze', ' yellow urine', ' yellowing of eyes', ' yellowish skin', 'itching']

symp_to_umlsV2(possible_symp)

[['C0000737'],
 ['C0205161', 'C0025344'],
 ['C0018834'],
 ['C0205178'],
 ['C0392747', 'C0234425'],
 ['C0003467'],
 ['C0004604'],
 ['C0224086'],
 ['C0333128'],
 ['C0005682', 'C2364135'],
 ['C0005758'],
 ['C0005767', 'C0038056'],
 ['C0018932'],
 ['C0344232', 'C0700135', 'C0042789'],
 ['C0013404'],
 ['C0439657', 'C0027342'],
 ['C0009938'],
 ['C0085624', 'C0042034'],
 ['C0008031'],
 ['C0085593'],
 ['C0424740', 'C0016504'],
 ['C0009421'],
 ['C0700148'],
 ['C0009806'],
 ['C0549178', 'C0042036'],
 ['C0549178', 'C0037383'],
 ['C0010200'],
 ['C0026821'],
 ['C0332582', 'C0042036'],
 ['C0011175'],
 ['C0011570'],
 ['C0011991'],
 [],
 ['C0012359', 'C0000726'],
 ['C0012833'],
 ['C0011682', 'C0877487'],
 ['C0442800', 'C0040132'],
 ['C0442802', 'C0020175'],
 [],
 ['C0241889'],
 ['C0039231'],
 ['C0015672'],
 ['C0302908', 'C0020488'],
 ['C0018520', 'C0042036'],
 ['C0018681'],
 ['C0205250', 'C0015967'],
 ['C0019559'],
 ['C0019664', 'C0001948'],
 ['C0205217', 'C0003618'],
 ['C0013395'],
 ['C0333348', 'C00

In [71]:
possible_symp = ['itching', 'skin rash', 'nodal skin eruptions', 'dischromic  patches', 'continuous sneezing', 'shivering', 'chills', 'watering from eyes', 'stomach pain', 'acidity', 'ulcers on tongue', 'vomiting', 'cough', 'chest pain', 'yellowish skin', 'nausea', 'loss of appetite', 'abdominal pain', 'yellowing of eyes', 'burning micturition', 'spotting  urination', 'passage of gases', 'internal itching', 'indigestion', 'muscle wasting', 'patches in throat', 'high fever', 'extra marital contacts', 'fatigue', 'weight loss', 'restlessness', 'lethargy', 'irregular sugar level', 'blurred and distorted vision', 'obesity', 'excessive hunger', 'increased appetite', 'polyuria', 'sunken eyes', 'dehydration', 'diarrhoea', 'breathlessness', 'family history', 'mucoid sputum', 'headache', 'dizziness', 'loss of balance', 'lack of concentration', 'stiff neck', 'depression', 'irritability', 'visual disturbances', 'back pain', 'weakness in limbs', 'neck pain', 'weakness of one body side', 'altered sensorium', 'dark urine', 'sweating', 'muscle pain', 'mild fever', 'swelled lymph nodes', 'malaise', 'red spots over body', 'joint pain', 'pain behind the eyes', 'constipation', 'toxic look (typhos)', 'belly pain', 'yellow urine', 'receiving blood transfusion', 'receiving unsterile injections', 'coma', 'stomach bleeding', 'acute liver failure', 'swelling of stomach', 'distention of abdomen', 'history of alcohol consumption', 'fluid overload', 'phlegm', 'blood in sputum', 'throat irritation', 'redness of eyes', 'sinus pressure', 'runny nose', 'congestion', 'loss of smell', 'fast heart rate', 'rusty sputum', 'pain during bowel movements', 'pain in anal region', 'bloody stool', 'irritation in anus', 'cramps', 'bruising', 'swollen legs', 'swollen blood vessels', 'prominent veins on calf', 'weight gain', 'cold hands and feets', 'mood swings', 'puffy face and eyes', 'enlarged thyroid', 'brittle nails', 'swollen extremeties', 'abnormal menstruation', 'muscle weakness', 'anxiety', 'slurred speech', 'palpitations', 'drying and tingling lips', 'knee pain', 'hip joint pain', 'swelling joints', 'painful walking', 'movement stiffness', 'spinning movements', 'unsteadiness', 'pus filled pimples', 'blackheads', 'scurring', 'bladder discomfort', 'foul smell of urine', 'continuous feel of urine', 'skin peeling', 'silver like dusting', 'small dents in nails', 'inflammatory nails', 'blister', 'red sore around nose', 'yellow crust ooze']
possible_symp_umls = sorted(symp_to_umls(possible_symp))

['C0000726', 'C0000737', 'C0000737', 'C0001948', 'C0003123', 'C0003126', 'C0003461', 'C0003467', 'C0003618', 'C0003862', 'C0004093', 'C0004093', 'C0004604', 'C0005682', 'C0005758', 'C0005767', 'C0005841', 'C0005847', 'C0006625', 'C0007004', 'C0007859', 'C0008031', 'C0009421', 'C0009806', 'C0009938', 'C0010200', 'C0011175', 'C0011570', 'C0011682', 'C0011991', 'C0012359', 'C0012833', 'C0013395', 'C0013404', 'C0013604', 'C0014448', 'C0014653', 'C0015230', 'C0015230', 'C0015385', 'C0015385', 'C0015392', 'C0015392', 'C0015392', 'C0015392', 'C0015392', 'C0015392', 'C0015450', 'C0015672', 'C0015967', 'C0016504', 'C0017110', 'C0018681', 'C0018834', 'C0018932', 'C0019080', 'C0019559', 'C0019664', 'C0020175', 'C0020488', 'C0021485', 'C0021853', 'C0022107', 'C0023380', 'C0024204', 'C0025344', 'C0025874', 'C0026649', 'C0026649', 'C0026649', 'C0026821', 'C0026845', 'C0027342', 'C0027342', 'C0027342', 'C0027497', 'C0028429', 'C0028429', 'C0028754', 'C0030193', 'C0030193', 'C0030193', 'C0030193', 'C0

In [136]:
#d is the output dictionary from the NLP
def pipeline(d):
    symp = d['symptoms']
    output_vector = [0] * len(possible_symp_umls)
    for i in range(len(symp)+2):
      random.shuffle(symp)
      umls_vector = symp_to_umls(symp)
      for word in umls_vector:
          if word in possible_symp_umls:
              index = possible_symp_umls.index(word)
              output_vector[index] = 1
    return output_vector
  
  


In [130]:
#loads rasan first predicting model
new_model = tf.keras.models.load_model('/content/drive/My Drive/ML/model_config')


In [141]:
#diseases dict
diseases=['(vertigo) Paroymsal  Positional Vertigo', 'AIDS', 'Acne', 'Alcoholic hepatitis', 'Allergy', 'Arthritis', 'Bronchial Asthma', 'Cervical spondylosis', 'Chicken pox', 'Chronic cholestasis', 'Common Cold', 'Dengue', 'Diabetes ', 'Dimorphic hemmorhoids(piles)', 'Drug Reaction', 'Fungal infection', 'GERD', 'Gastroenteritis', 'Heart attack', 'Hepatitis B', 'Hepatitis C', 'Hepatitis D', 'Hepatitis E', 'Hypertension ', 'Hyperthyroidism', 'Hypoglycemia', 'Hypothyroidism', 'Impetigo', 'Jaundice', 'Malaria', 'Migraine', 'Osteoarthristis', 'Paralysis (brain hemorrhage)', 'Peptic ulcer diseae', 'Pneumonia', 'Psoriasis', 'Tuberculosis', 'Typhoid', 'Urinary tract infection', 'Varicose veins', 'hepatitis A']

In [183]:
def decode_one_hot(pred):
  index = np.argmax(np.array(pred), axis=-1)
  print(diseases[index]+ "     Probability :"+str(pred[index]*100))
  
 


In [196]:
def prediction(nlp_output):
  decode_one_hot(new_model.predict([pipeline(nlp_output)])[0])



In [198]:
d={'symptoms': ['skin_rash', 'blackheads'], 'sex': 'Female', 'age': '26'}

prediction(d)

1/1 [==============================] - 0s 40ms/step
Acne     Probability :42.990899085998535
